## Conexión JVP

In [7]:
import mysql.connector
import pandas as pd
from datetime import datetime

db = mysql.connector.connect(host="192.168.0.12",    #  host
                     user="operario",         #  usuario
                     passwd="",  #  password
                     db="dbp8100")        #  nombre de la base de datos

cur = db.cursor()


In [8]:
# Extraer Fórmulas activas
# Datos mysql
cur.execute('''
            SELECT NroID, Codigo, Nombre, VT FROM dbp8100.formulas;''')
resultados = cur.fetchall()
# Obtener los nombres de las columnas
columnas = [desc[0] for desc in cur.description]

# Crear un DataFrame de pandas
df = pd.DataFrame(resultados, columns=columnas)

# Cerrar el cursor y la conexión
cur.close()
db.close()

# Mostrar el DataFrame
print(df)

      NroID  Codigo                                Nombre  VT
0        35    None           FIBROMIXER 300  19% A.A. -1   0
1        36     3-K     VACA LECHERA 16% anulada no hacer   0
2        39  07-009  DEST. PRECOZ MEL. 0%A.A CASC DE SOJA   0
3        40  08-020          INICIADOR MELAZA (21 %A.A.)    0
4        41    09-H            C.M. URUGUAY REFORZADO 10%   0
...     ...     ...                                   ...  ..
1156   1254  09-141     CRE. MELAZA 10% A.A. (Los Roanos)   1
1157   1255  FP0028      MEZCLA AUTOCONSUMO (SUÑE MIGUEL)   1
1158   1256  FP0029             DESTETE PRECOZ I  (Gepsa)   1
1159   1257  FP0030                     TERNEROS  (Gepsa)   1
1160   1258  FP0031           CONCENTRADO VACUNOS (Gepsa)   1

[1161 rows x 4 columns]


In [20]:
import psycopg2
import pandas as pd

# Replace with your actual database credentials
DB_HOST = "192.168.0.12"
DB_NAME = "db_mto"
DB_USER = "apsa"
DB_PASSWORD = "Apsa2812p."
DB_PORT = "5432" # Often 5432


try:
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(host=DB_HOST, database=DB_NAME, user=DB_USER, password=DB_PASSWORD, port=DB_PORT)
    cur = conn.cursor()

    # Load the DataFrame (assuming it's already loaded from Parte_diario.ipynb)
    #  Replace 'df' with the actual name of your DataFrame if different.
    #df = pd.read_csv("your_data.csv") #Example if you need to load it from a file.


    # Create the table if it doesn't exist.  Adjust column types as needed.
    # This assumes your DataFrame has columns named 'column1', 'column2', etc.
    # Replace with the actual column names and data types from your DataFrame.
    create_table_query = """
        CREATE TABLE IF NOT EXISTS formulaS (
            nroid INT PRIMARY KEY,
            codigo VARCHAR(20),
            nombre VARCHAR(100),
            vt SMALLINT
        );
    """
    cur.execute(create_table_query)


    # Insert data from the DataFrame into the table
    for index, row in df.iterrows():
        # Construct the INSERT query dynamically.  Be cautious about SQL injection!
        #  This example assumes all columns are strings. Adjust as needed for other types.
        columns = ','.join(df.columns)
        placeholders = ','.join(['%s'] * len(df.columns))
        insert_query = f"""
            INSERT INTO formulas ({columns}) VALUES ({placeholders});
        """
        cur.execute(insert_query, tuple(row))

    # Commit the changes and close the connection
    conn.commit()
    print("Data inserted successfully!")

except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL:", error)
finally:
    if conn:
        cur.close()
        conn.close()
        print("PostgreSQL connection is closed")



Data inserted successfully!
PostgreSQL connection is closed
